In [1]:
from batchedCartpole import CartPole
import sys
sys.path.append("../ES")
from ES_NeuralGraph import NeuralGraph
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

C:\Users\Alec\AppData\Local\Temp\ipykernel_23956\599906760.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
n_inp = 4
n_out = 3
n_hid = 8
n_reward = 1

n_nodes = n_inp + n_out + n_hid + n_reward

connections = torch.Tensor([[i, j] for i in range(n_nodes) for j in range(n_nodes)]).long()

graph = NeuralGraph(n_nodes, n_inp + n_reward, n_out, connections, ch_n=4, ch_e=4, clamp_mode="hard", max_value=5, init_mode="trainable", set_nodes=True, aggregation="mean", 
                     use_label=False, n_models=1)

env = CartPole()

optimizer = torch.optim.Adam(graph.parameters(), lr=1e-3)

In [9]:
EPOCHS = 10_000
STEPS = 200

BATCH_SIZE = 64
TIME = 1
DT = .5

bar = tqdm(range(EPOCHS))

for _ in bar:
    with torch.no_grad():
        graph.init_vals(nodes=True, edges=True, batch_size=BATCH_SIZE)
        graph.generate_epsilons(BATCH_SIZE)
        state = env.initialize(BATCH_SIZE)
        reward = torch.zeros(BATCH_SIZE)
        optimizer.zero_grad()

        regret = torch.zeros(BATCH_SIZE)

        for _ in range(STEPS):
            action = graph.forward(torch.cat([state, reward.unsqueeze(-1)], axis=-1), TIME, DT, apply_once=False, nodes=True, edges=True).argmax(-1)
            reward = env.step(action)
            regret -= reward

        graph.estimate_grads(regret)

        optimizer.step()

        bar.set_postfix({f"Avg. Regret:":regret.mean().item()})

  0%|          | 0/10000 [00:00<?, ?it/s]